# Predicting most Efficient Interventions for Life Span Increase
#### Data Literacy at University of Tübingen, Winter 2023/24

---
&copy; Daniel Flat, Jackson Harmon, Eric Nazarenus, Aline Bittler 2023. [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

Template by Philipp Hennig, 2023. [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

In [77]:
import os
import pandas as pd

/tmp/ipykernel_15332/3571106454.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [91]:
# Group Constants
MALE = 'male'
FEMALE = 'female'
SEX_CATEGORIES = [MALE, FEMALE]

# Intervention Constants
CONTROL_GROUP = 'None'
ALL_GROUP = 'All'
ONE_REMOVED_PREFIX = 'no_'

# Dataset Contsants
DATASET_DIRECTORY = os.path.join('..', 'dat', 'RMR_data')

In [92]:
dataset_paths = os.listdir(DATASET_DIRECTORY)

def convert_filename_to_key(filename, category):
    category_removed = filename[len(category) + 1:]
    return os.path.splitext(category_removed)[0]

def create_dataset_mapping(dataset_directory, categories):
    dataset_mappings = {}
    
    for category in categories:
        category_path = {} 
        for path in dataset_paths:
            full_path = os.path.join(dataset_directory, path)
    
            if category == path[:len(category)]:
                key = convert_filename_to_key(path, category)
                category_path[key] = full_path
        dataset_mappings[category] = category_path
    
    return dataset_mappings

In [114]:
def is_one_removed_intervention(key):
    if len(key) < 3:
        return False

    return key[:len(ONE_REMOVED_PREFIX)] == ONE_REMOVED_PREFIX

def extract_single_interventions(dataset_mapping: dict):
    single_interventions = []
    for key in dataset_mapping.keys():
        if key == ALL_GROUP:
            continue
        elif key == CONTROL_GROUP:
            continue
        elif is_one_removed_intervention(key):
            continue
        else:
            single_interventions.append(key)
    
    return single_interventions

def create_one_removed_key(key: str, intervention_list: []) -> []:
    removed_key = key[len(ONE_REMOVED_PREFIX):]

    removed_intervention_list = []
    for intervention in intervention_list:
        if intervention != removed_key:
            removed_intervention_list.append(removed_key)

    return removed_intervention_list

def create_intervention_list_from_group(key: str, intervention_list: []):
        if key == ALL_GROUP:
            return intervention_list
        elif key == CONTROL_GROUP:
            return []
        elif is_one_removed_intervention(key):
            return create_one_removed_key(key, intervention_list)
        else:
            return key

def create_standard_key_from_intevention_list(interventions):
    return ','.join(interventions)

In [115]:
def load_csv(path):
    return pd.read_csv(
                        path,
                        skiprows=0,
                        sep=',',
                        engine='python',
                        decimal='.',
                        dtype={'x': 'float64', 'y': 'float64'}
            )


def create_dataset_by_category(category_path: dict):
    datasets = {}
    for key, path in category_path.items():
        datasets[key] = load_csv(path)
    return datasets

# create_dataset_by_category(female_dataset_paths)
dataset_mapping = create_dataset_mapping(DATASET_DIRECTORY, SEX_CATEGORIES)
female_dataset_paths = dataset_mapping[FEMALE]

SINGLE_INTERVENTIONS = extract_single_interventions(female_dataset_paths)

create_standard_key_from_intevention_list(SINGLE_INTERVENTIONS)
# female_dataset_paths

'HSCs,Gal-Nav,mTERT,Rapamycin'

In [ ]:
def roundNumbers(df):
    df = df.apply(lambda x:round(x, ROUNDING_DIGITS))
    return df

def addColumn(data_frame, col_name, col_value):
    data_frame[col_name] = col_value

# Round the numbers of the x and y columns
female_df = roundNumbers(female_df)

# Add the columns for the first group
addColumn(female_df, 'sex', 'female')
addColumn(female_df, 'category', 'None')

female_df